# Predict Survival on Titanic  

In this notebook, I will fit and try to predict survival in the Titanic using the Titanic dataset which is available on [kaggle](https://www.kaggle.com/c/titanic/data).

In [2]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import os

## Load dataset

In [3]:
def load_dataset(path = 'titanic'):
    dataset_path = os.path.join(path, 'train.csv')
    return pd.read_csv(dataset_path)

In [4]:
df = load_dataset()

*Since test.csv does not have labels column on it, I won't be using it. gender_submission.csv is an example with dummy values, so it will be ignored as well.*

In [ ]:
#